In [6]:
#Introduction and Library Imports

# Pricing Options using Fast Fourier Transform (FFT) with real-life stock data
# This notebook calculates option prices using FFT and compares them with real-world data.

# Importing Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# Configure pandas for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Display settings for matplotlib
#plt.style.use('seaborn')



In [7]:
#Fetching Real-Life Stock Data

# Fetch stock data using yfinance
ticker = "AAPL"                 # Stock ticker (e.g., "AAPL" for Apple)
start_date = "2023-01-01"       # Start date
end_date = "2023-12-31"         # End date

stock_data = yf.download(ticker, start=start_date, end=end_date)
stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']]
stock_data['Daily Return'] = stock_data['Close'].pct_change()
stock_data.dropna(inplace=True)

# Display the stock data
print(f"Stock data for {ticker}:")
stock_data.tail(10)

# Set Parameters

# Parameters for Option Pricing
S0 = stock_data['Close'].iloc[-1]  # Initial stock price (last closing price)
K = S0 * 1.05                     # Strike price: 5% above the current price
r = 0.03                          # Risk-free interest rate (example: 3%)
q = 0.01                          # Dividend yield (example: 1%)
T = 0.5                           # Time to maturity in years (6 months)
volatility = 0.2                  # Assumed stock volatility (20%)

# FFT Parameters
n = 12                            # FFT exponent: Determines resolution (2^n points)
N = 2 ** n                        # Number of FFT steps
eta = 0.25                        # Integration step size
alpha = 1.0                       # Damping factor to stabilize the Fourier transform

print(f"Option Pricing Parameters for {ticker}:")
pd.DataFrame({
    'Parameter': [
        'Initial Stock Price (S0)', 'Strike Price (K)', 'Risk-Free Rate (r)', 
        'Dividend Yield (q)', 'Time to Maturity (T)', 'FFT Steps (N)', 
        'Integration Step Size (eta)', 'Damping Factor (alpha)'
    ],
    'Value': [S0, K, r, q, T, N, eta, alpha]
})

[*********************100%***********************]  1 of 1 completed

Stock data for AAPL:
Option Pricing Parameters for AAPL:


,Parameter,Value
0,Initial Stock Price (S0),Ticker AAPL 191.59 Name: 2023-12-29 00:00:00...
1,Strike Price (K),Ticker AAPL 201.17 Name: 2023-12-29 00:00:00...
2,Risk-Free Rate (r),0.03
3,Dividend Yield (q),0.01
4,Time to Maturity (T),0.50
5,FFT Steps (N),4096
6,Integration Step Size (eta),0.25
7,Damping Factor (alpha),1.00


In [8]:
# Characteristic Function Calculation

# Black-Scholes characteristic function
u = np.arange(N) * eta - (alpha + 1) * 1j
drift = np.log(S0) + (r - q - 0.5 * volatility**2) * T
diffusion = volatility * np.sqrt(T)
phi = np.exp(1j * u * drift - 0.5 * (diffusion * u)**2)

# Explanation of formula:
# The characteristic function is computed as:
# φ(u) = exp(1j * u * drift - 0.5 * (diffusion * u)^2)
# Where:
# - drift = ln(S0) + (r - q - 0.5 * σ²) * T
# - diffusion = σ * sqrt(T)

# FFT Computation

# Integration step for FFT
discount_factor = np.exp(-r * T)
weights = eta * np.ones(N)
weights[0] = eta / 2  # Adjust the first weight for numerical accuracy

# Apply FFT
psi = phi / ((alpha + 1j * u) * (alpha + 1 + 1j * u))
fft_values = np.fft.fft(discount_factor * weights * psi)

# Strike prices and option prices
delta_k = (2 * np.pi / N) / eta
log_strikes = np.log(K) + np.arange(N) * delta_k
option_prices = (np.exp(-alpha * log_strikes) / np.pi) * np.real(fft_values)

# Convert log-strikes to strikes
strikes = np.exp(log_strikes)
# Fetch Real Option Chain Data

# Fetch real-world option chain data
expiration_date = "2024-01-19"  # Example expiration date
option_chain = yf.Ticker(ticker).option_chain(expiration_date)
calls = option_chain.calls[['strike', 'lastPrice']]

# Format real-world data
real_strikes = calls['strike']
real_prices = calls['lastPrice']

print(f"Real Option Chain for {ticker} (Expiration: {expiration_date}):")
calls.head()

# Plot and Compare Prices

# Plot FFT option prices vs real-world option prices
plt.figure(figsize=(12, 6))
plt.plot(strikes, option_prices, label="FFT Option Prices", color="blue", linestyle='-')
plt.scatter(real_strikes, real_prices, color="red", label="Real Option Prices", alpha=0.6)
plt.axvline(x=K, color="green", linestyle="--", label="Strike Price (K)")
plt.title(f"Option Price Comparison for {ticker}")
plt.xlabel("Strike Price")
plt.ylabel("Option Price")
plt.legend()
plt.grid(True)
plt.show()

ValueError: Length of values (4096) does not match length of index (1)